In [1]:

import numpy as np
import os
import cv2
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras import layers
from keras.models import Sequential
import matplotlib.pyplot as plt
import matplotlib.image as img
from keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Input, Lambda ,Dense ,Flatten ,Dropout
from keras.models import Model
import keras
# define path to dataset
path = "/kaggle/input/sign-language-dataset/ImagePro"
path2 = "/kaggle/input/myimagepro/myImagePro"



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

## Preparations

In [ ]:
#pip install opencv-python

In [2]:
pip install cvzone

  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6-py3-none-any.whl size=26342 sha256=e956942599754e9085c08277113eb32ccd32005a5e3613df8c0d486866404ffe
  Stored in directory: /root/.cache/pip/wheels/1e/c1/41/09fa72fe85489b6cf8a6e93ea78c0e98ba80b010466ee23fca
Successfully built cvzone
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 35.3 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install gdown

In [ ]:
#https://drive.google.com/file/d/1e8bDeSfhrS1cRpEt8LO13GFrTlKEHrN7/view?usp=drivesdk
!gdown --id 1e8bDeSfhrS1cRpEt8LO13GFrTlKEHrN7

In [ ]:
#https://drive.google.com/file/d/1Jn4cZWXOwoVBXeCizxmI3H6xYhrQyIsP/view?usp=drivesdk
!gdown --id 1Jn4cZWXOwoVBXeCizxmI3H6xYhrQyIsP

Download the 1st person dataset

In [ ]:
!unzip "/kaggle/working/Copy of Dataset.zip" -d "/kaggle/working/ImagePro"
!unzip "/kaggle/working/ImagePro.zip" -d "/kaggle/working/ImagePro2"

In [ ]:
from cvzone.HandTrackingModule import HandDetector
import cv2

In [ ]:
detector = HandDetector(detectionCon=0.8, maxHands=2)
img = cv2.imread("/kaggle/working/ImagePro2/7/2023-09-02_11-52-52.jpg")
hands, img_ = detector.findHands(img.copy())  # with draw

In [ ]:
plt.imshow(img_)

## Getting Data

In [ ]:
files=os.listdir(path)
# list of files in path
# sort path from A-Y
files.sort ()

# print to see list
print(files)


In [ ]:

# create list of image and label
image_array = []
label_array = []

# Loop through each file in files
for i in range(len(files)):
    # List of images in each folder
    sub_files = os.listdir(path + "/" + files[i])
    
    # Let's check the length of each folder
    #print(len(sub_files))
    
    # Loop through each sub folder
    for j in range(len(sub_files)):
        # Path of each image
        # Example: imagepro/A/image_name.jpg
        file_path = path + "/" + files[i] + "/" + sub_files[j]
        
        # Read each image
        image = cv2.imread(file_path)
        
        # Resize image to 96x96
        image = cv2.resize(image, (96, 96))
        
        # Convert BGR image to RGB image
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        image_array.append(image)
        label_array.append(i)


In [ ]:
# convert list to array
image_array = np.array(image_array)
label_array = np.array(label_array, dtype="float")

# split the dataset into test and train
#from sklearn.model_selection inport train_test_split

In [ ]:
##---New Data loading method
#main_dir = "/kaggle/working/ImagePro"
main_dir = "/kaggle/input/myimagepro/myImagePro"
uniq_labels = sorted(os.listdir(main_dir))
#print(uniq_labels)
lab_dic={}
for v,k in enumerate(uniq_labels):
    lab_dic[k.lower()] = v
#lab_dic["0"] = 40
print(lab_dic)

def file_gen(directory):
    images = []
    labels = []
    #directory = train_dir2
    train_files = sorted(os.listdir(directory))
    #print(train_files)
    for label in train_files:
        for idx, image_nm in enumerate(os.listdir(directory + "/" + label)):
            if idx in range(200):
               filepath = directory + "/" + label + "/" + image_nm
               #print(image_nm)
               #print(idx)
               try: 
                    img = cv2.resize(cv2.imread(filepath), (96,96))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
               except: 
                    continue               
               images.append(img)
               #plt.imshow(image)
               tag = lab_dic[label.lower()]
               labels.append(tag)
               #print(tag)
        
    return images, labels 

In [ ]:
detector = HandDetector(detectionCon=0.8, maxHands=2)
'''
##---New Data loading method
#main_dir = "/kaggle/working/ImagePro"
main_dir = "/kaggle/input/sign-language-dataset/ImagePro"
uniq_labels = sorted(os.listdir(main_dir))
#print(uniq_labels)
lab_dic={}
for v,k in enumerate(uniq_labels):
    lab_dic[k.lower()] = v
lab_dic["0"] = 40
print(lab_dic)

'''
def file_gen_(directory):
    images = []
    labels = []
    #directory = train_dir2
    train_files = sorted(os.listdir(directory))
    #print(train_files)
    for label in train_files:
        if label == "Blank":
            continue
        if label == "10":
            continue
        for idx, image_nm in enumerate(os.listdir(directory + "/" + label)):            
            if idx in range(10):
               filepath = directory + "/" + label + "/" + image_nm
               #print(image_nm)
               #print(idx)
               try: 
                    img = cv2.imread(filepath)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    hands, img_ = detector.findHands(img.copy())  # with draw
               except: 
                    continue
               if hands:
                    print("image detected")
                    hand = hands[0]
                    x1, y1 = hand["bbox"][0], hand["bbox"][1]
                    x2, y2 = x1 + hand["bbox"][2], y1 + hand["bbox"][3]
                    padding = 50  # adjust this value to increase/decrease padding
                    x1 -= padding
                    y1 -= padding
                    x2 += padding
                    y2 += padding
                    cropped_img = img[max(0, y1):min(y2, img.shape[0]), max(0, x1):min(x2, img.shape[1])]    
                    cropped_img = cv2.resize(cropped_img, (64,64))
                    #cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
                    tag = lab_dic[label.lower()]
                    images.append(cropped_img)
                    #plt.imshow(image)
                    labels.append(tag)
                    #print(tag)
               else:
                    print("no image detected")
    return images, labels 

In [ ]:
#create a data list
data_list = []
label_list = []


#read from dir1
#train_dir1 = "/kaggle/working/ImagePro"
train_dir2 = "/kaggle/input/myimagepro/myImagePro"
#images_1, labels_1 = file_gen(train_dir1)
images_2, labels_2 = file_gen(train_dir2)
#images_4, labels_4 = file_gen(train_dir4)

#data_list.extend(images_1)
#label_list.extend(labels_1)
data_list.extend(images_2)
label_list.extend(labels_2)
#data_list.extend(images_4)
#label_list.extend(labels_4)

#del(images_1)
#del(labels_1)
del(images_2)
del(labels_2)
#del(images_4)
#del(labels_4)

image_array = np.asarray(data_list)
label_array = np.asarray(label_list)

In [ ]:
##____for test


data_list_test = []
label_list_test = []


#read from dir1
test_dir1 = "/kaggle/input/synthetic-asl-numbers/Train_Nums"
#images_test, labels_test = file_gen(test_dir1)
images_test, labels_test = file_gen_(test_dir1)

data_list_test.extend(images_test)
label_list_test.extend(labels_test)

del(images_test)
del(labels_test)

image_array_test = np.asarray(data_list_test)
label_array_test = np.asarray(label_list_test)

y_cat_test = keras.utils.to_categorical(label_array_test)


In [ ]:
len(image_array_test)

In [ ]:
image_array_test[5]

In [ ]:
plt.imshow(image_array_test[80])

In [ ]:
# output spliting size
X_train, X_test, y_train, y_test = train_test_split(image_array, label_array, test_size=0.15)

# X_train will have 85% of images
# X_test will have 15% of inages

In [ ]:
len(X_train)

In [ ]:
del image_array
del label_array

**converting to categorical for vgg16**

In [ ]:
import keras

y_train_cat = keras.utils.to_categorical(y_train)
y_test_cat = keras.utils.to_categorical(y_test)
#y_eval_cat = keras.utils.to_categorical(y_eval)

In [ ]:
y_train_cat.shape

**Fist model (EfficientNetDB)**

In [ ]:
from keras import layers, callbacks, utils, applications, optimizers
from keras.models import Sequential, Model, load_model

model = Sequential()

# Add pretrained models to Sequential model
# I will use EfficientNetB0 pretrained model. You can try a different model.
pretrained_model = tf.keras.applications.EfficientNetB0(input_shape=(96,96,3), include_top=False)

#don't train existing weights for vgg16
#for layer in classifier_vgg16.layers:
    #layer.trainable = False

model.add(pretrained_model)

# Add pooling to the model
model.add(layers.GlobalAveragePooling2D())

# Add dropout to the model
# We add dropout to increase accuracy by reducing overfitting
model.add(layers.Dropout(0.3))

# Finally, add a dense layer as the output
model.add(layers.Dense(1))

# Print the model summary
model.summary()
model.compile(optimizer="adam", loss="mae", metrics=["mae"])


In [ ]:

# Create a checkpoint to save the best model
checkpoint_path = "trained_model/model"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor="val_mae",
    mode="auto",
    save_best_only=True,
    save_weights_only=True
)

# Learning rate reducer
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    patience=5,
    factor=0.9,
    monitor="val_mae",
    mode="auto",
    cooldown=0,
    verbose=1,
    min_lr=1e-6
)



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model3
model3 = tf.keras.Sequential()

# Convolutional layers
model3.add(Conv2D(32, (3, 3), activation='relu', input_shape=(96, 96, 3)))
model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Conv2D(64, (3, 3), activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Conv2D(128, (3, 3), activation='relu'))
model3.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the feature maps
model3.add(Flatten())

# Dense (fully connected) layers
model3.add(Dense(256, activation='relu'))
model3.add(Dropout(0.5))  # Dropout for regularization

model3.add(Dense(128, activation='relu'))
model3.add(Dropout(0.5))  # Dropout for regularization

# Output layer
model3.add(Dense(10, activation='softmax'))  # 10 output classes

# Compile the model3
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model3 summary
model3.summary()


**Model 2 (VGG16)**

In [ ]:
#Initialising vgg16 
classifier_vgg16 = VGG16(input_shape= (96,96,3),include_top=False,weights='imagenet')

#don't train existing weights for vgg16
for layer in classifier_vgg16.layers:
    layer.trainable = False

classifier1 = classifier_vgg16.output#head mode
classifier1 = Flatten()(classifier1)#adding layer of flatten
classifier1 = Dense(units=256, activation='relu')(classifier1)
classifier1 = Dropout(0.6)(classifier1)
classifier1 = Dense(units=10, activation='softmax')(classifier1)

model2 = Model(inputs = classifier_vgg16.input , outputs = classifier1)
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model2.summary()

In [ ]:
y_test_cat.shape

In [ ]:
# start training model
epochs = 50
batch_size = 32
# select batch size according to your graphic card
# X_train, X_test, Y_train, Y_test
#history = model2.fit(
history = model.fit(
    X_train,
    y_train,
    #validation_data=(X_test, y_test),
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[model_checkpoint, reduce_lr]
)
# before training you can delete image_array and label_array to increase ram memory



In [ ]:
# start training model
epochs = 50
batch_size = 32
# select batch size according to your graphic card
# X_train, X_test, Y_train, Y_test
history = model2.fit(
    X_train,
    y_train_cat,
    #validation_data=(X_test, y_test_cat),
    batch_size=batch_size,
    epochs=epochs,
    #callbacks=[model_checkpoint, reduce_lr]
)
# before training you can delete image_array and label_array to increase ram memory



In [ ]:
# start training model
epochs = 5
batch_size = 32
# select batch size according to your graphic card
# X_train, X_test, Y_train, Y_test
history = model3.fit(
    X_train,
    y_train_cat,
    #validation_data=(X_test, y_test_cat),
    batch_size=batch_size,
    epochs=epochs,
    #callbacks=[model_checkpoint, reduce_lr]
)
# before training you can delete image_array and label_array to increase ram memory



In [ ]:
print(X_train.shape)
print(y_train_cat.shape)

Evaluation on normal test data

In [ ]:
#score = model.evaluate(x = X_test, y = y_test, verbose = 0)
#print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = model3.evaluate(x = X_test, y = y_test_cat, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

Evaluation on a differrent test data

In [ ]:
#score = model.evaluate(x = image_array_test, y = y_cat_test, verbose = 0)
#print('Accuracy for test images:', round(score[1]*100, 3), '%')
score = model2.evaluate(x = image_array_test, y = y_cat_eval, verbose = 0)
print('Accuracy for evaluation images:', round(score[1]*100, 3), '%')

In [ ]:
predictions = model.predict(X_test, batch_size=2)
print(predictions[:10])
print(y_test[:10])

In [ ]:
predictions = model2.predict(X_test, batch_size=2)
print(predictions[:10])
print(y_test[:10])

In [ ]:
lab_dic

In [ ]:
image_array= []

# Read each image
image = cv2.imread("/kaggle/working/ImagePro/ImagePro/5/2023-08-30_18-27-17.jpg")
        
# Resize image to 96x96
image = cv2.resize(image, (96, 96))
        
# Convert BGR image to RGB image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
image_array.append(image)
#label_array.append(i)

# convert list to array
image_array = np.array(image_array)

predictions = model3.predict(image_array, batch_size=2)
pred = np.argmax(predictions)
print(pred)
plt.imshow(image)

In [ ]:
im = X_test[90]
predictions = model2.predict(np.array([im]), batch_size=2)
pred = np.argmax(predictions)
print(pred)
plt.imshow(im)

In [ ]:
detector = HandDetector(detectionCon=0.8, maxHands=2)
image = cv2.imread("/kaggle/working/ImagePro/ImagePro/2/2023-08-30_18-21-32.jpg")
hands, img = detector.findHands(image.copy())  # with draw

if hands:
    print("image detected")
    hand = hands[0]
    x1, y1 = hand["bbox"][0], hand["bbox"][1]
    x2, y2 = x1 + hand["bbox"][2], y1 + hand["bbox"][3]
    padding = 50  # adjust this value to increase/decrease padding
    x1 -= padding
    y1 -= padding
    x2 += padding
    y2 += padding
    cropped_img = image[max(0, y1):min(y2, img.shape[0]), max(0, x1):min(x2, img.shape[1])]    
    #cv2_imshow(cropped_img)
else:
    print("No image detected")
    cropped_img = image

image_array= []

# Read each image
#image = cv2.imread("/kaggle/input/mydata-alpha-2long/20230830_125715.jpg")

image = cropped_img
# Resize image to 96x96
image = cv2.resize(image, (96, 96))
        
# Convert BGR image to RGB image
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
image_array.append(image)
#label_array.append(i)

# convert list to array
image_array = np.array(image_array)
predictions = model2.predict(image_array, batch_size=2)
pred = np.argmax(predictions)
print(pred)
plt.imshow(image)
#plt.imshow(cropped_img)